Tässä työkirjassa esitettävät tiedot perustuvat kirjan [Applied Predictive Modeling](https://link.springer.com/book/10.1007/978-1-4614-6849-3) kappaleeseen Data Pre-processing.

# Multikollineaarisuus

Multikollineaarisuus tarkoittaa tilannetta, jossa usean muuttujan välillä esiintyy vahvaa korrelaatiota; toisin sanoen nämä muuttujat kertovat kokolailla saman asian, mutta jokainen hieman eri tavalla.

## Mitä haittaa

Kun mallintamisessa tavoitteena on hyvän yleistyskyvyn omaava mahdollisimman yksinkertainen malli, niin päällekkäistä tietoa sisältävät muuttujat lisäävät mallin monimutkaisuutta kuitenkaan tuomatta lisäinformaatiota.

Oppaan kirjoittajat nostavat esille lineaarisen regression kaltaiset tekniikat, joilla multikollineaariset aineistot voivat heikentää mallin suorituskykyä ja luotettavuutta.

## Kuinka tunnistaa

Korrelaatiomatriisin visuaalinen tarkastelu voi paljastaa keskenään korreloivia ryhmiä. R-kielen [corrplot-funktion](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) avulla korrelaatiomatriisi voidaan tulostaa siten, että keskenään korreloivat muuttujat esitellään vierekkäin.

Multikollineaarisuutta voidaan selvittää myös [pääkomponenttianalyysin avulla](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) (PCA). Mikäli muutama pääkomponentti selittää suurimman osan aineiston varianssista, viittaa tämä siihen, että muuttujien välillä esiintyy multikollineaarisuutta. Komponenttien loadings arvojen tarkastelu voi auttaa tunnistamaan mistä muuttujista on kyse.

## Kuinka päästä eroon

Regressioanalyysiin on kehitetty erilaisia menetelmiä multikollineaarisuuden tunnistamiseen. 

Esimerkiksi kun tietyn aineiston perustella on laadittu regressiomalli, niin VIF-mittarin avulla voi arvioida kärsiikö malli multikollineaarisuuten liittyvistä ongelmista ja pyrkiä tarvittaessa tunnistamaan sekä poistamaan kollineaariset muuttujat.

VIF-mittarin avulla saatua tietoa ei kuitenkaan (yleensä) voi suoraan hyödyntää toisenlaisia malleja laadittaessa.

[Pääkomponenttianalyysi](https://fi.wikipedia.org/wiki/P%C3%A4%C3%A4komponenttianalyysi) tarjoaa tehokkaan menetelmän multikollineaarisuuden poistamiseen. 

PCA:n haittana on selitettävyyden vaikeutuminen. Kun alkuperäisistä muuttujista siirrytään pääkomponenttien käyttöön, kausaalisten yhteyksien selittäminen hankaloituu merkittävästi. Lisäksi PCA tutkii ainoastaan selittävien muuttujien varianssia, eikä huomioi selittävien ja itsenäisen muuttujien välistä vuorovaikutusta. Ratkaisuksi on kehitetty [PLS:n](https://en.wikipedia.org/wiki/Partial_least_squares_regression) kaltaisia menetelmiä, jotka paremmin huomioivat edellä mainitun yhteyden.

Kirja esittelee myös heuristisen menetelmän, joka mahdollisimman vähäisellä muuttujien poistamisella huolehtii siitä, että jäljelle jäävien muuttujien keskinäiset korrelaatiot jäävät halutun raja-arvon alle. Vaikka algoritmi operoikin kahden muuttujan välisillä korrelaatiolla, menetelmän avulla on mahdollista parantaa kollineaarisuudesta kärsivien mallien toimintaa.

Algoritmi toimii seuraavalla tavalla:

1. Lasketaan muuttujien välinen korrelaatiomatriisi.
2. Valitaan muuttujat, joiden välillä vallitsee voimakkain korrelaatio. Oletetaan, että näiden muuttujien nimet ovat A ja B.
3. Määritetään A:n ja muiden (jäljellä olevien) muuttujien välinen keskinäinen korrelaatio. Lasketaan vastaava arvo myös muuttujalle B.
4. Mikäli A:lla keskimääräisen korrelaation arvo on suurempi, poistetaan se aineistosta. Vaihtoehtoisessa tapauksessa poistetaan muuttuja B.
5. Toistetaan vaiheita 2 - 4 niin kauan kunnes yhdenkään jäljellä olevan muuttujaparin keskimääräinen korrelaatio ei ylitä raja-arvoa.

Kirjassa käytetään [caret-paketista](https://www.rdocumentation.org/packages/caret/versions/6.0-94/topics/findCorrelation) löytyvää R kielistä toteutusta em. algoritmista ([findCorrelation](https://github.com/topepo/caret/blob/master/pkg/caret/R/findCorrelation.R)).

Lähdekoodin tarkastelu paljastaa, että ohjelmakoodissa huomoidaan muuttujien määrän. Mikäli määrä ylittää 100 kappaleen rajan, suoritetaan yleisluontoisempi muuttujien karsinta.

Käydään tässä dokumentissä läpi kuinka tarkempi laskenta saadaan toteutettua python koodin avulla. 

Ladataan käytettävät paketit

In [1]:
import numpy as np
import pandas as pd

Aliohjelma,  joka tarkistaa, että matriisi on [symmetrinen](https://fi.wikipedia.org/wiki/Symmetrinen_matriisi).

In [2]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

Aliohjelma, joka mahdollisimman vähäisellä muuttujien karsimisella varmistaa, että jäljelle jäävien muuttujien välillä ei esiinny multikollineaarisuutta.

In [4]:
def findCorrelation(x, cutoff = 0.9):

    removed = []

    # Varmistetaan, että aineiston tietotyypit ovat float tyyppisiä
    if (x.dtypes.values[:, None] == ['int64', 'int32', 'int16', 'int8']).any():
            x = x.astype(float)

    # Irroitetaan arvot Numpy-taulukkoon
    _x = x.values

    # Napataan rivien lkm muistiin
    varnum = _x.shape[0]

    # Varmistetaan, että matriisi symmetrinen
    if check_symmetric(_x) == False:
        print("Ei oo symmetrinen")
        return None
    
    # Tämän tarkistuksen merkitystä en ymmärtänyt
    # - jos matriisin koko (1,1), niin kaatuu jo symmetrisyyteen?
    if varnum == 1:
        print("only one variable given")
        return None
    
    # Jatketaan lukujen absoluuttisilla arvoilla
    _x = np.abs(_x)


    # tyhjennetään diagonaaliakseli
    # - oletus on, että korrelaatiomatriisi sisältää float-arvoja....
    # - na.arvoja ei huomioda keskiarvoja laskettaessa
    np.fill_diagonal(_x, np.nan)

    # Muuttujien järjestys korrelaatioarvojen keskiarvon perustella
    maxAbsCorOrder = np.nanmean(_x, axis = 0).argsort()[::-1]

    for i in range(len(maxAbsCorOrder) - 1):
        iIndx = maxAbsCorOrder[i]

        # Onko sarake jo aiemmin poistettu
        if iIndx in removed:
            continue

        for j in range(i + 1, len(maxAbsCorOrder)):

            jIndx = maxAbsCorOrder[j]

            # Onko jompikumpi jo poistettu
            if iIndx in removed or jIndx in removed:
                continue
            
            # Ylittääkö muuttujien välinen korrelaatio raja-arvon
            if _x[iIndx][jIndx] > cutoff:
                    
                mn1 = np.nanmean(_x[iIndx,:])
                mn2 = np.nanmean(_x[:,jIndx])

                if mn1 > mn2:

                    removed.append(iIndx)
                    _x[iIndx,:] = np.nan
                    _x[:,iIndx] = np.nan
                else:

                    removed.append(jIndx)
                    _x[jIndx,:] = np.nan
                    _x[:,jIndx] = np.nan


    return list(x.columns[removed])


Määritellään testiaineisto:

In [5]:

np.random.seed([3,1415])
df = pd.DataFrame(
    np.random.randint(10, size=(10, 10)),
    columns=list('ABCDEFGHIJ'))

Lasketaan ja tulostetaan korrelaatiomatriisi

In [10]:

corr = df.corr()
# corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)
corr.style.background_gradient(cmap='coolwarm', axis=None).format(precision=3)

,A,B,C,D,E,F,G,H,I,J
A,1.000,0.218,0.424,-0.117,-0.172,-0.161,-0.112,0.352,0.126,-0.057
B,0.218,1.000,0.098,-0.080,-0.182,0.070,0.332,0.122,-0.342,0.174
C,0.424,0.098,1.000,-0.077,-0.407,-0.119,-0.425,0.548,0.197,0.340
D,-0.117,-0.080,-0.077,1.000,-0.048,-0.287,0.268,0.022,-0.450,0.107
E,-0.172,-0.182,-0.407,-0.048,1.000,0.470,0.005,-0.376,-0.188,-0.864
F,-0.161,0.070,-0.119,-0.287,0.470,1.000,-0.620,-0.666,-0.077,-0.545
G,-0.112,0.332,-0.425,0.268,0.005,-0.620,1.000,0.217,-0.402,0.072
H,0.352,0.122,0.548,0.022,-0.376,-0.666,0.217,1.000,0.496,0.593
I,0.126,-0.342,0.197,-0.450,-0.188,-0.077,-0.402,0.496,1.000,0.396
J,-0.057,0.174,0.340,0.107,-0.864,-0.545,0.072,0.593,0.396,1.000


Lasketaan poistettavat sarakkeet, ylärajan ollessa 0.5:

In [15]:
hc = findCorrelation(corr, cutoff=0.5)
print(hc)

['H', 'J', 'G']
